In [74]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torchvision import transforms
from sklearn.model_selection import train_test_split
import numpy as np

# epoch = one forward and backward pass of ALL training samples
# batch_size = number of training samples used in one forward/backward pass
# number of iterations = number of passes, each pass (forward+backward) using [batch_size] number of sampes
# e.g : 100 samples, batch_size=20 -> 100/20=5 iterations for 1 epoch


In [77]:
class DataSet():

    def __init__(self):
        
        self.x_data = torch.rand(100,4,5,3)
        y = np.random.randint(0, 4, 100)
        self.y_data =torch.from_numpy(y).long()
        self.n_samples = 100

    # support indexing such that dataset[i] can be used to get i-th sample
    def __getitem__(self, index):
        return self.x_data[index], self.y_data[index]

    # we can call len(dataset) to return the size
    def __len__(self):
        return self.n_samples

In [ ]:
class SimpleNN(nn.Module):
    def __init__(self):
        super(SimpleNN, self).__init__()
        self.fc1 = nn.Linear(4 * 5 * 3, 10)
        self.fc2 = nn.Linear(10, 10)
        self.fc3 = nn.Linear(10, 10)
        self.fc4 = nn.Linear(10, 4)

    def forward(self, x):
        x = x.view(-1, 4 * 5 * 3)
        x = torch.relu(self.fc1(x))
        x = torch.relu(self.fc2(x))
        x = torch.relu(self.fc3(x))
        x = self.fc4(x)
        return x

In [78]:
dataset = DataSet()

In [79]:
# Split the dataset into training, validation, and test sets
train_data, test_data = train_test_split(dataset, test_size=0.1, random_state=42)


In [80]:
# Create DataLoaders
train_loader = DataLoader(train_data, batch_size=10, shuffle=True)
test_loader = DataLoader(test_data, batch_size=10, shuffle=True)


In [81]:
# Define the model, loss function, and optimizer
model = SimpleNN()
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [85]:
# Train the model
n_total_steps = len(train_loader)
for epoch in range(2):
    for i, (images, labels) in enumerate(train_loader):
        
        # Forward pass
        outputs = model(images)
        loss = criterion(outputs, labels)
        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        print (f'Epoch [{epoch+1}/{2}], Step [{i+1}/{n_total_steps}], Loss: {loss.item():.4f}')


Epoch [1/2], Step [1/9], Loss: 1.3774
Epoch [1/2], Step [2/9], Loss: 1.3963
Epoch [1/2], Step [3/9], Loss: 1.4041
Epoch [1/2], Step [4/9], Loss: 1.3912
Epoch [1/2], Step [5/9], Loss: 1.3815
Epoch [1/2], Step [6/9], Loss: 1.4039
Epoch [1/2], Step [7/9], Loss: 1.3725
Epoch [1/2], Step [8/9], Loss: 1.3968
Epoch [1/2], Step [9/9], Loss: 1.3855
Epoch [2/2], Step [1/9], Loss: 1.3887
Epoch [2/2], Step [2/9], Loss: 1.3893
Epoch [2/2], Step [3/9], Loss: 1.3955
Epoch [2/2], Step [4/9], Loss: 1.3886
Epoch [2/2], Step [5/9], Loss: 1.3836
Epoch [2/2], Step [6/9], Loss: 1.3841
Epoch [2/2], Step [7/9], Loss: 1.3911
Epoch [2/2], Step [8/9], Loss: 1.3837
Epoch [2/2], Step [9/9], Loss: 1.3874


In [89]:
# Test the model
# In test phase, we don't need to compute gradients (for memory efficiency)
with torch.no_grad():
    n_correct = 0
    n_samples = 0
    for images, labels in test_loader:
        outputs = model(images)
        # max returns (value ,index)
        _, predicted = torch.max(outputs.data, 1)
        n_samples += labels.size(0)
        n_correct += (predicted == labels).sum().item()

    acc = 100.0 * n_correct / n_samples
    print(f'Accuracy: {acc} %')

Accuracy: 50.0 %
